In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
ad_col = ['adid', 'advert_id', 'orderid', 'campaign_id', 'creative_id','creative_tp_dnf', 'creative_type', 'advert_name','advert_industry_inner_split_0', 'advert_industry_inner_split_1']
ad_cate_col = ['creative_is_jump', 'creative_is_download', 'creative_is_js','creative_is_voicead', 'creative_has_deeplink']
medim_col = ['app_cate_id', 'app_id', 'inner_slot_id', 'app_paid']
user_col = ['user_tags_0', 'user_tags_1', 'user_tags_2', 'user_tags_3', 'user_tags_4','user_tags_5', 'user_tags_6', 'user_tags_7', 'user_tags_8', 'user_tags_9','user_tags_10', 'user_tags_11', 'user_tags_12', 'user_tags_13','user_tags_14', 'user_tags_15', 'user_tags_16', 'user_tags_17','user_tags_18', 'user_tags_19', 'user_tags_20', 'user_tags_21','user_tags_22', 'user_tags_23', 'user_tags_24', 'user_tags_25','user_tags_26', 'user_tags_27', 'user_tags_28', 'user_tags_29','user_tags_30', 'user_tags_31', 'user_tags_32', 'user_tags_33','user_tags_34', 'user_tags_35', 'user_tags_36', 'user_tags_37']
context_col = ['city', 'province', 'carrier', 'devtype', 'make', 'model', 'nnt', 'os','osv', 'os_name']

cont_col = ['creative_width', 'creative_height']
by_col = ['instance_id', 'time']
target = ['click']

bool_col = ['creative_is_jump','creative_is_download','creative_is_js','creative_is_voicead','creative_has_deeplink','app_paid']
zip_col = ['city','province','carrier','devtype','make','model','nnt','os','osv','os_name','adid','advert_id','orderid','campaign_id','creative_id','creative_tp_dnf','app_cate_id','app_id','inner_slot_id','creative_type','advert_name','advert_industry_inner_split_0','advert_industry_inner_split_1']

cate_col = bool_col + zip_col
full_col = ad_col + ad_cate_col + medim_col + user_col + context_col + cont_col + by_col + target
use_col = list(set(full_col) - {'instance_id', 'time', 'click'})
len(use_col)

69

In [3]:
df = pd.read_csv('./feature/feature_nonoise/full_feature.csv', nrows=1)
feas = list(set(df.columns) - set(full_col))
len(feas)

4855

## lgb筛选

In [4]:
import lightgbm as lgb
from sklearn.cross_validation import train_test_split

C:\Users\ZERO\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
cv_params =  {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'min_data_in_leaf': 30,
        'num_leaves': 31,
        'max_depth': 5,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'seed': 2018
        }

In [6]:
from sklearn.metrics import log_loss

In [11]:
# df = pd.read_csv('./feature/feature_nonoise/full_feature_train.csv', usecols=full_col)
X_train, X_val, y_train, y_val = train_test_split(df[df.click != -1], df[df.click != -1].click, test_size=0.3, random_state=2018)
train_data = lgb.Dataset(X_train[use_col], label=X_train['click'], categorical_feature=cate_col)
bst = lgb.train(cv_params, train_data, verbose_eval=0, num_boost_round=100)
y_pred = bst.predict(X_val[use_col])

print('origin', log_loss(y_val, y_pred))

origin 0.41869205346362776


In [7]:
feature_performance = {}

for i in range(49):
    df = pd.read_csv('./feature/feature_nonoise/full_feature.csv', usecols=full_col + feas[i*101:(i+1)*101])
    X_train, X_val, y_train, y_val = train_test_split(df[df.click != -1], df[df.click != -1].click, test_size=0.3, random_state=2018)
    
    for fea in tqdm_notebook(feas[i*101:(i+1)*101]):
        tmp_feas = use_col + [fea]
        train_data = lgb.Dataset(X_train[tmp_feas], label=X_train['click'], categorical_feature=cate_col)
        bst = lgb.train(cv_params, train_data, verbose_eval=0, num_boost_round=100)
        y_pred = bst.predict(X_val[tmp_feas])
        
        feature_performance[fea] = log_loss(y_val, y_pred)
        print(fea, feature_performance[fea])
        del train_data
        _ = gc.collect()
    del df
    del X_train
    del X_val
    _ = gc.collect()

A Jupyter Widget

creative_type_user_tags_10pos/nag_rate 0.41851544062654034
city_user_tags_15pos/nag_rate 0.4185898892415617
user_tags_5_user_tags_32pos/nag_rate 0.4185135818117083
campaign_id_user_tags_6pos/nag_rate 0.4186082234358246
make_user_tags_1pos/nag_rate 0.4184616676248129
app_paid_user_tags_2pos/nag_rate 0.4186243839201845
user_tags_24_pos_sample 0.4186754860491542
user_tags_33_ospos/nag_rate 0.4185653913654682
user_tags_19_user_tags_0pos/nag_rate 0.4186129040999167
creative_id_user_tags_20pos/nag_rate 0.41855485547871674
advert_industry_inner_split_1_adidpos/nag_rate 0.41856910886560855
user_tags_21_creative_is_jspos/nag_rate 0.4185904868717633
user_tags_17_user_tags_8pos/nag_rate 0.4185492580532179
user_tags_20_creative_typepos/nag_rate 0.41857815984658275
creative_is_voicead_user_tags_10pos/nag_rate 0.4186238249363964
creative_id_user_tags_27pos/nag_rate 0.418519232292437
model_user_tags_10pos/nag_rate 0.4182616579396336
user_tags_1_ospos/nag_rate 0.41859312023405026
user_tags_36_province

A Jupyter Widget

user_tags_1_user_tags_10pos/nag_rate 0.4186005161072192
user_tags_10_app_paidpos/nag_rate 0.4186238249363964
user_tags_23_user_tags_21pos/nag_rate 0.41863259941006853
user_tags_25_user_tags_12pos/nag_rate 0.4185321065426981
user_tags_4_creative_idpos/nag_rate 0.4185359404819119
os_citypos/nag_rate 0.41862008279619894
city_user_tags_33pos/nag_rate 0.41851366852081967
creative_is_jump_user_tags_7pos/nag_rate 0.418638358729859
user_tags_27_user_tags_5pos/nag_rate 0.41848802619067826
user_tags_25_pos_sample 0.4186118327070817
advert_industry_inner_split_0_user_tags_7pos/nag_rate 0.4185119676497982
click_user_tags_35_mean 0.41854659099667785
user_tags_17_creative_is_downloadpos/nag_rate 0.41857104234883974
user_tags_3_user_tags_9pos/nag_rate 0.41853121976235
user_tags_9_osvpos/nag_rate 0.41857310369378037
user_tags_33_user_tags_3pos/nag_rate 0.4185491455762395
user_tags_24_user_tags_10pos/nag_rate 0.4186036574880861
creative_id_ospos/nag_rate 0.41852767726089246
user_tags_1_advert_industry_

A Jupyter Widget

...

user_tags_7_user_tags_34pos/nag_rate 0.4186325583380225
user_tags_37_user_tags_2pos/nag_rate 0.41855701422105573
user_tags_1_user_tags_33pos/nag_rate 0.4186051038937353
user_tags_35_os_namepos/nag_rate 0.4185903460754191
user_tags_3_user_tags_24pos/nag_rate 0.41858165325781943
creative_tp_dnf_user_tags_20pos/nag_rate 0.41856832820562945
user_tags_21_ospos/nag_rate 0.4185623033707802
nnt_user_tags_27pos/nag_rate 0.4185433012551255
user_tags_4_user_tags_22pos/nag_rate 0.41848792283955505
adid_single_last_time_sub 0.4186314076001675
creative_tp_dnf_user_tags_34pos/nag_rate 0.41873150231771417
user_tags_17_advert_namepos/nag_rate 0.4186075027599758
make_ospos/nag_rate 0.41852110659011044
user_tags_37_user_tags_0pos/nag_rate 0.4185919730028792
user_tags_25_user_tags_14pos/nag_rate 0.4185321065426981
user_tags_9_user_tags_19pos/nag_rate 0.4185795828882155
user_tags_17_user_tags_6pos/nag_rate 0.4185168407666649
user_tags_2_user_tags_37pos/nag_rate 0.41855701422105573
user_tags_8_osvpos/nag_ra

A Jupyter Widget

user_tags_14_inner_slot_idpos/nag_rate 0.41851665615981176
os_name_user_tags_15pos/nag_rate 0.41851937585838317
creative_id_advert_idpos/nag_rate 0.418583191817999
click_user_tags_1_count 0.4186243839201845
user_tags_36_user_tags_35pos/nag_rate 0.41853961671595896
user_tags_21_orderidpos/nag_rate 0.4185219650931987
user_tags_30_orderidpos/nag_rate 0.4185406170549245
orderid_creative_tp_dnfpos/nag_rate 0.4186059072387608
creative_tp_dnf_nag_sample 0.418567456914486
user_tags_13_user_tags_7pos/nag_rate 0.4185656468752603
creative_is_voicead_user_tags_29pos/nag_rate 0.41859870308090125
user_tags_37_ospos/nag_rate 0.41855343735244815
user_tags_27_app_idpos/nag_rate 0.4186380369531316
user_tags_15_app_idpos/nag_rate 0.4185564806652823
user_tags_21_user_tags_10pos/nag_rate 0.4185514595195689
user_tags_14_user_tags_31pos/nag_rate 0.4186244807412984
app_id_user_tags_6pos/nag_rate 0.4185398139244596
nnt_citypos/nag_rate 0.4185311544267706
user_tags_20_user_tags_17pos/nag_rate 0.4184857131073271

A Jupyter Widget

user_tags_21_user_tags_11pos/nag_rate 0.4185847143306923
user_tags_8_user_tags_3pos/nag_rate 0.4186105081628412
creative_has_deeplink_user_tags_13pos/nag_rate 0.41859870308090125
user_tags_37_os_namepos/nag_rate 0.41855343735244815
user_tags_35_advert_idpos/nag_rate 0.41868149697411694
user_tags_31_user_tags_28pos/nag_rate 0.4185462138651157
user_tags_25_creative_has_deeplinkpos/nag_rate 0.4186118327070817



In [32]:
feature_performance

{'user_tags_3_creative_idpos/nag_rate': 0.4185174472509271,
 'advert_id_user_tags_4pos/nag_rate': 0.4185032930178218,
 'user_tags_19_pos_sample': 0.4185966228083274,
 'user_tags_5_campaign_idpos/nag_rate': 0.41859061841251144,
 'click_user_tags_18_mean_sub': 0.005533815383754852,
 'creative_type_user_tags_17pos/nag_rate': 0.41853064080354163,
 'user_tags_17_user_tags_1pos/nag_rate': 0.41846513746302955,
 'creative_is_voicead_ospos/nag_rate': 0.41855909148696624,
 'user_tags_33_user_tags_16pos/nag_rate': 0.4185133076317628,
 'user_tags_1_carrierpos/nag_rate': 0.41861907410820587,
 'make_user_tags_26pos/nag_rate': 0.4186009219713481,
 'make_user_tags_3pos/nag_rate': 0.418497944460506,
 'user_tags_30_user_tags_2pos/nag_rate': 0.4185231436219127,
 'user_tags_5_devtypepos/nag_rate': 0.41856331505755384,
 'user_tags_6_user_tags_22pos/nag_rate': 0.4185821933878253,
 'user_tags_32_carrierpos/nag_rate': 0.41856839716948663,
 'advert_id_nag_sample': 0.41860850653017395,
 'nnt_app_paidpos/nag_rat

In [8]:
sorted(feature_performance.items(), key=lambda x:x[1], reverse=False)[:300]

[('noclick', 0.005532430973582788),
 ('inner_slot_id_modelpos/nag_rate', 0.41778038663167416),
 ('model_inner_slot_idpos/nag_rate', 0.41778038663167416),
 ('creative_type_modelpos/nag_rate', 0.4178097658270491),
 ('model_creative_typepos/nag_rate', 0.4178097658270491),
 ('model_app_idpos/nag_rate', 0.41795928390285214),
 ('app_id_modelpos/nag_rate', 0.41795928390285214),
 ('creative_tp_dnf_modelpos/nag_rate', 0.41811759529923026),
 ('model_creative_tp_dnfpos/nag_rate', 0.41811759529923026),
 ('model_app_cate_idpos/nag_rate', 0.4181374823042454),
 ('app_cate_id_modelpos/nag_rate', 0.4181374823042454),
 ('model_user_tags_21pos/nag_rate', 0.41817018408345),
 ('user_tags_21_modelpos/nag_rate', 0.41817018408345),
 ('advert_name_modelpos/nag_rate', 0.4181911223170414),
 ('model_advert_namepos/nag_rate', 0.4181911223170414),
 ('model_user_tags_4pos/nag_rate', 0.41819832103533905),
 ('user_tags_4_modelpos/nag_rate', 0.41819832103533905),
 ('model_creative_is_jumppos/nag_rate', 0.41820239916167

In [11]:
good_fea = [i[0] for i in filter(lambda x:(x[1] > 0.417) & (x[1] <= 0.4185) , sorted(feature_performance.items(), key=lambda x:x[1], reverse=False))]
len(list(good_fea))

686

In [12]:
list(good_fea)

['inner_slot_id_modelpos/nag_rate',
 'model_inner_slot_idpos/nag_rate',
 'creative_type_modelpos/nag_rate',
 'model_creative_typepos/nag_rate',
 'model_app_idpos/nag_rate',
 'app_id_modelpos/nag_rate',
 'creative_tp_dnf_modelpos/nag_rate',
 'model_creative_tp_dnfpos/nag_rate',
 'model_app_cate_idpos/nag_rate',
 'app_cate_id_modelpos/nag_rate',
 'model_user_tags_21pos/nag_rate',
 'user_tags_21_modelpos/nag_rate',
 'advert_name_modelpos/nag_rate',
 'model_advert_namepos/nag_rate',
 'model_user_tags_4pos/nag_rate',
 'user_tags_4_modelpos/nag_rate',
 'model_creative_is_jumppos/nag_rate',
 'creative_is_download_modelpos/nag_rate',
 'creative_is_jump_modelpos/nag_rate',
 'model_creative_is_downloadpos/nag_rate',
 'user_tags_15_modelpos/nag_rate',
 'model_user_tags_15pos/nag_rate',
 'model_user_tags_37pos/nag_rate',
 'user_tags_37_modelpos/nag_rate',
 'inner_slot_id_osvpos/nag_rate',
 'osv_inner_slot_idpos/nag_rate',
 'model_user_tags_31pos/nag_rate',
 'user_tags_31_modelpos/nag_rate',
 'mode

## time feature

In [3]:
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

C:\Users\ZERO\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df = pd.read_csv('./feature/full_feature.csv', usecols=full_col)

df.shape

(348536, 72)

In [5]:
df_time = pd.read_csv('./feature/time_feature.csv')

df_time.shape

(1041674, 50)

In [6]:
df = df.merge(df_time, on=['instance_id'], how='left')
df.shape

(348536, 121)

In [8]:
feas = [i for i in df_time.columns if 'time_sub' in i]
feas

['adid_single_next_time_sub',
 'adid_single_last_time_sub',
 'advert_id_single_next_time_sub',
 'advert_id_single_last_time_sub',
 'orderid_single_next_time_sub',
 'orderid_single_last_time_sub',
 'campaign_id_single_next_time_sub',
 'campaign_id_single_last_time_sub',
 'creative_id_single_next_time_sub',
 'creative_id_single_last_time_sub',
 'creative_tp_dnf_single_next_time_sub',
 'creative_tp_dnf_single_last_time_sub',
 'creative_type_single_next_time_sub',
 'creative_type_single_last_time_sub',
 'advert_name_single_next_time_sub',
 'advert_name_single_last_time_sub',
 'advert_industry_inner_split_0_single_next_time_sub',
 'advert_industry_inner_split_0_single_last_time_sub',
 'advert_industry_inner_split_1_single_next_time_sub',
 'advert_industry_inner_split_1_single_last_time_sub',
 'app_cate_id_single_next_time_sub',
 'app_cate_id_single_last_time_sub',
 'app_id_single_next_time_sub',
 'app_id_single_last_time_sub',
 'inner_slot_id_single_next_time_sub',
 'inner_slot_id_single_la

In [10]:
cv_params =  {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'min_data_in_leaf': 30,
        'num_leaves': 31,
        'max_depth': 5,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'seed': 2018
        }

In [11]:
feature_performance = {}

X_train, X_val, y_train, y_val = train_test_split(df[df.click != -1], df[df.click != -1].click, test_size=0.3, random_state=2018)
    
for fea in tqdm_notebook(feas):
    tmp_feas = use_col + [fea]
    train_data = lgb.Dataset(X_train[tmp_feas], label=X_train['click'], categorical_feature=cate_col)
    bst = lgb.train(cv_params, train_data, verbose_eval=0, num_boost_round=100)
    y_pred = bst.predict(X_val[tmp_feas])
    
    feature_performance[fea] = log_loss(y_val, y_pred)
    print(fea, feature_performance[fea])
    del train_data
    _ = gc.collect()

A Jupyter Widget

adid_single_next_time_sub 0.41865094741443254
adid_single_last_time_sub 0.4186502676951463
advert_id_single_next_time_sub 0.4186684142040774
advert_id_single_last_time_sub 0.4185649546054312
orderid_single_next_time_sub 0.4186916573501788
orderid_single_last_time_sub 0.4186057702156854
campaign_id_single_next_time_sub 0.4186798466941968
campaign_id_single_last_time_sub 0.4186945061876008
creative_id_single_next_time_sub 0.4186647707331511
creative_id_single_last_time_sub 0.4186676671340476
creative_tp_dnf_single_next_time_sub 0.41865962686426417
creative_tp_dnf_single_last_time_sub 0.41861376070964057
creative_type_single_next_time_sub 0.4186129528530918
creative_type_single_last_time_sub 0.4186147465733885
advert_name_single_next_time_sub 0.4186746631737688
advert_name_single_last_time_sub 0.4186015234346885
advert_industry_inner_split_0_single_next_time_sub 0.418704727659768
advert_industry_inner_split_0_single_last_time_sub 0.418680915158096
advert_industry_inner_split_1_single_next